# Customer Revenue Prediction

## PyTorch LSTM Model
*Machine Learning Nanodegree Program | Capstone Project*

---

In this notebook I will be creating a PyTorch LSTM model and compare it with the baseline model I created earlier.

### Overview:
- Reading the data
- Preparing the tensors for the PyTorch Model
- Initializing the LSTM model
- Training the model with the train dataset
- Validating the model using the val dataset
- Predict the revenue for customer in test dataset
- Visualizing the results
- Compare the results with the baseline model
- Saving the results to a csv 

First, import the relevant libraries into notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sagemaker
import boto3

from os import path
from sklearn.metrics import mean_squared_error

%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.10f' % x)

In [2]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()

prefix = 'sagemaker/capstone-project'

print(bucket)

sagemaker-us-east-1-201308845573


Set the various paths for the training, validation, test files and storing the baseline results

In [4]:
data_dir = '../datasets'

if not path.exists(data_dir):
    raise Exception('{} directory not found.'.format(data_dir))

train_file = '{}/{}'.format(data_dir, 'train.zip')
print('\nTrain file: {}'.format(train_file))

val_file = '{}/{}'.format(data_dir, 'val.zip')
print('\nValidation file: {}'.format(val_file))

pred_val_file = '{}/{}'.format(data_dir, 'lstm_pred_val.zip')
print('\nValidation Prediction file: {}'.format(pred_val_file))

test_file = '{}/{}'.format(data_dir, 'test.zip')
print('\nTest file: {}'.format(test_file))

pred_test_file = '{}/{}'.format(data_dir, 'lstm_pred_test.zip')
print('\nTest Prediction file: {}'.format(pred_test_file))

imp_features_file = '{}/{}'.format(data_dir, 'lstm_importances-01.png')
print('\nImportant Features file: {}'.format(imp_features_file))

input_s3_train_file = sagemaker_session.upload_data(path=train_file, bucket=bucket, key_prefix=prefix)
print('\nInput data S3 Train file: {}'.format(input_s3_train_file))

input_s3_val_file = sagemaker_session.upload_data(path=val_file, bucket=bucket, key_prefix=prefix)
print('\nInput data S3 Val file: {}'.format(input_s3_val_file))

input_s3_dir = 's3://{}/{}'.format(bucket, prefix)
print('\nInput data S3 directory: {}'.format(input_s3_dir))



Train file: ../datasets/train.zip

Validation file: ../datasets/val.zip

Validation Prediction file: ../datasets/lstm_pred_val.zip

Test file: ../datasets/test.zip

Test Prediction file: ../datasets/lstm_pred_test.zip

Important Features file: ../datasets/lstm_importances-01.png

Input data S3 Train file: s3://sagemaker-us-east-1-201308845573/sagemaker/capstone-project/train.zip

Input data S3 Val file: s3://sagemaker-us-east-1-201308845573/sagemaker/capstone-project/val.zip

Input data S3 directory: s3://sagemaker-us-east-1-201308845573/sagemaker/capstone-project


In [ ]:
empty_check = []

for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('\nTest passed!')

Method to load the dataset from the files

In [5]:
def load_data(zip_path):
    df = pd.read_csv(
        zip_path,
        dtype={'fullVisitorId': 'str'},
        compression='zip'
    )
    
    [rows, columns] = df.shape

    print('Loaded {} rows with {} columns from {}.'.format(
        rows, columns, zip_path
    ))
    
    return df

Load the train, validation and test datasets.

In [6]:
%%time

train_df = load_data(train_file)
val_df = load_data(val_file)
test_df = load_data(test_file)

print()

Loaded 765707 rows with 26 columns from ../datasets/train.zip.
Loaded 137946 rows with 26 columns from ../datasets/val.zip.
Loaded 804684 rows with 25 columns from ../datasets/test.zip.

CPU times: user 13.9 s, sys: 482 ms, total: 14.4 s
Wall time: 14.2 s


In [ ]:
# remove this at the end

import torch

train_df.head()

total_rows = train_df.shape[0] - (train_df.shape[0] % 512)

train_df = train_df.iloc[:total_rows]

train_y = np.log1p(train_df['totals.transactionRevenue'].values)
train_y = torch.from_numpy(train_y).float().squeeze()

train_X = train_df.drop(['totals.transactionRevenue', 'fullVisitorId'], axis=1).values
train_X = torch.from_numpy(train_X).float()

train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1])

train_X.shape

train_df.head()

In [16]:
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)

estimator = PyTorch(
    entry_point='lstm_train.py',
    source_dir='../models/pytorch/',
    role=role,
    output_path=output_path,
    sagemaker_session=sagemaker_session,
    framework_version='1.2',
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    hyperparameters={
        'input_dim': 24,
        'n_layers': 1,
        'batch-size': 512,
        'hidden_dim': 512
    }
)

In [17]:
estimator.fit({'train': input_s3_dir})

2020-01-29 02:48:44 Starting - Starting the training job...
2020-01-29 02:48:45 Starting - Launching requested ML instances......
2020-01-29 02:49:50 Starting - Preparing the instances for training......
2020-01-29 02:50:54 Downloading - Downloading input data......
2020-01-29 02:52:09 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-01-29 02:52:10,376 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-01-29 02:52:10,379 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-29 02:52:10,392 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-01-29 02:52:13,440 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-01-29 02:52:13,783 sagemaker-containers INFO     Module lstm_train does not provide a setup.py. 
Generat

Epoch: 6/10... Step: 8000... Loss: 0.5888871436... Val Loss: 1.8448269857
Epoch: 6/10... Step: 8500... Loss: 1.1519187000... Val Loss: 1.8006097012
Epoch: 7/10... Step: 9000... Loss: 0.0268457722... Val Loss: 1.7812685543
Epoch: 7/10... Step: 9500... Loss: 0.5862691539... Val Loss: 1.8014457610
Epoch: 7/10... Step: 10000... Loss: 1.1518902157... Val Loss: 1.8012037630
Epoch: 8/10... Step: 10500... Loss: 0.0331645429... Val Loss: 1.7820918578
Epoch: 8/10... Step: 11000... Loss: 0.5928613699... Val Loss: 1.7876186079
Epoch: 8/10... Step: 11500... Loss: 1.1550816457... Val Loss: 1.7706644787
Epoch: 9/10... Step: 12000... Loss: 0.0352508879... Val Loss: 1.7946457750
Epoch: 9/10... Step: 12500... Loss: 0.5896079801... Val Loss: 1.7811959346
Epoch: 9/10... Step: 13000... Loss: 1.1517768794... Val Loss: 1.7749186867
Epoch: 10/10... Step: 13500... Loss: 0.0405484863... Val Loss: 1.7686244375
Epoch: 10/10... Step: 14000... Loss: 0.6023859428... Val Loss: 1.7709673091
Epoch: 10/10... Step: 14500

In [18]:
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
    model_data=estimator.model_data,
    role=role,
    framework_version='1.2',
    entry_point='lstm_predict.py',
    source_dir='../models/pytorch'
)

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------------------!

In [19]:
def get_batches(dataset, size=1024):
    for i in range(0, len(dataset), size):  
        yield dataset[i:(i + size)]
        
def predict_batch(predictor, dataset):
    pred_arr = np.array([])
    
    for next_batch in get_batches(dataset):
        temp_pred = predictor.predict(next_batch)
        
        pred_arr = np.append(pred_arr, temp_pred)
    
    return pred_arr

In [20]:
total_rows = val_df.shape[0] - (val_df.shape[0] % 1024)

# val_df = val_df.iloc[:total_rows]

val_id = val_df['fullVisitorId'].values

val_y = val_df['totals.transactionRevenue'].values
val_X = val_df.drop(['totals.transactionRevenue', 'fullVisitorId'], axis=1).values

val_X = val_X.reshape(val_X.shape[0], 1, val_X.shape[1])

pred_val = predict_batch(predictor, val_X)

pred_val[pred_val < 0] = 0

pred_val_data = {
    'fullVisitorId': val_id,
    'transactionRevenue': val_y,
    'predictedRevenue': np.expm1(pred_val)
}

pred_val_df = pd.DataFrame(pred_val_data)

pred_val_df = pred_val_df.groupby('fullVisitorId')['transactionRevenue', 'predictedRevenue'].sum().reset_index()

pred_val_df.head()

,fullVisitorId,transactionRevenue,predictedRevenue
0,0000062267706107999,0.0000000000,0.0000000000
1,0000085059828173212,0.0000000000,0.0000000000
2,000026722803385797,0.0000000000,0.0000000000
3,0000436683523507380,0.0000000000,19.0855369232
4,0000450371054833295,0.0000000000,0.0000000000


In [21]:
rsme_val = np.sqrt(
    mean_squared_error(
        np.log1p(pred_val_df['transactionRevenue'].values),
        np.log1p(pred_val_df['predictedRevenue'].values)
    )
)

print('\nRSME for validation data set: {:.10f}\n'.format(rsme_val))


RSME for validation data set: 1.8070251573



In [22]:
total_rows = test_df.shape[0] - (test_df.shape[0] % 1024)

# test_df = test_df.iloc[:total_rows]

test_id = test_df['fullVisitorId'].values
test_X = test_df.drop(['fullVisitorId'], axis=1).values

test_X = test_X.reshape(test_X.shape[0], 1, test_X.shape[1])

pred_test = predict_batch(predictor, test_X)

pred_test[pred_test < 0] = 0

pred_test_data = {
    'fullVisitorId': test_id,
    'predictedRevenue': np.expm1(pred_test)
}

pred_test_df = pd.DataFrame(pred_test_data)

pred_test_df = pred_test_df.groupby('fullVisitorId')['predictedRevenue'].sum().reset_index()

pred_test_df.head()

,fullVisitorId,predictedRevenue
0,0000000259678714014,19.0855369232
1,0000049363351866189,0.0000000000
2,0000053049821714864,0.0000000000
3,0000059488412965267,0.0000000000
4,0000085840370633780,0.0000000000


In [23]:
pred_val_df.to_csv(pred_val_file, index=False, compression='zip')

pred_test_df.to_csv(pred_test_file, index=False, compression='zip')

In [24]:
def delete_endpoint(predictor):
    try:
        predictor.delete_endpoint()
        print('Deleted {}'.format(predictor.endpoint))
    except: 
        print('Already deleted: {}'.format(predictor.endpoint))
        

delete_endpoint(predictor)

Deleted pytorch-inference-2020-01-29-03-10-29-341


In [25]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket)

bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '9236CDD7B058A268',
   'HostId': '6HvzLSyzryUHtS275dqhm1jULceIZW+ImY+NVTCOMWdQTUz4WOtDMtWJ3B2LIQsi0I9fOWG4tTI=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '6HvzLSyzryUHtS275dqhm1jULceIZW+ImY+NVTCOMWdQTUz4WOtDMtWJ3B2LIQsi0I9fOWG4tTI=',
    'x-amz-request-id': '9236CDD7B058A268',
    'date': 'Wed, 29 Jan 2020 03:26:22 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/capstone-project/pytorch-training-2020-01-29-02-44-23-008/debug-output/training_job_end.ts'},
   {'Key': 'sagemaker/capstone-project/pytorch-training-2020-01-29-02-48-43-855/debug-output/training_job_end.ts'},
   {'Key': 'sagemaker/capstone-project/pytorch-training-2020-01-29-02-48-43-855/output/model.tar.gz'},
   {'Key': 'pytorch-training-2020-01-29-01-44-11-934/source/sourcedir.tar.gz'},
   {'Key': 'sagemaker/capstone-project/pytor